In [1]:
import sys, os
sys.path.append(os.pardir)
import module.category as cg
import module.refinement as rm
import module.predict as pre
import module.result as rs
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
nation = 'FR'
videos= pd.read_csv('../youtube-new/'+nation+'videos.csv')

In [3]:
# publish_time, trending_date 날짜형식으로 변환
rm.retypeDate(videos)

# 'publish_time'이 [start, end]에 속하지 않는 데이터 행 삭제
start = min(videos['trending_date'])
end = max(videos['publish_time'])
rm.dropRows_dates(videos, 'publish_time', start, end)

# 에러 및 오류가 있는 데이터 행 삭제
# rm.dropRows_disabled(videos, 'comments_disabled') # views에 영향을 미치지 않음
# rm.dropRows_disabled(videos, 'ratings_disabled') # views에 영향을 미치지 않음
# rm.dropRows_disabled(videos, 'video_error_or_removed') # views에 영향을 미치지 않음

# 사용하지 않는 column 삭제
rm.dropColumns(videos, 'tags')
rm.dropColumns(videos, 'description')

In [4]:
language = 'French'
kedoo_data = cg.kedooRank(language)
result_data = cg.info_category()
list_category_id = cg.list_category()

In [5]:
unique = pre.init_unique(videos)
stat = pre.init_stat(unique, list_category_id)
predict_method = 'sum_total_views'
pre.score(result_data, stat, predict_method)
result_data

,category_TITLE,sum_total_views,rank
category_ID,,,
1,Film & Animation,598235933,6
2,Autos & Vehicles,35618600,13
10,Music,2651291836,1
15,Pets & Animals,32050554,15
17,Sports,1366663953,3
19,Travel & Events,5280316,16
20,Gaming,267110237,9
22,People & Blogs,878483088,5
23,Comedy,955594121,4


In [6]:
reliability_method = 2
rs.calc(result_data, 'rank',kedoo_data[language].values,reliability_method)
result_data

,category_TITLE,sum_total_views,rank,reliability
category_ID,,,,
1,Film & Animation,598235933,6,92.307692
2,Autos & Vehicles,35618600,13,80.000000
10,Music,2651291836,1,100.000000
15,Pets & Animals,32050554,15,83.333333
17,Sports,1366663953,3,33.333333
19,Travel & Events,5280316,16,84.615385
20,Gaming,267110237,9,64.285714
22,People & Blogs,878483088,5,86.666667
23,Comedy,955594121,4,83.333333


In [7]:
result=rs.resultType(result_data, 'rank','category_TITLE','sum_total_views','reliability')

In [8]:
print('video\t\t\t',nation)
print('language\t\t',language)
print('predict_method\t\t',predict_method)
print('reliability_method\t',reliability_method,'\n')
print('Total Reliability\t',round(np.mean(result_data['reliability']),2),'%')
result

video			 FR
language		 French
predict_method		 sum_total_views
reliability_method	 2 

Total Reliability	 82.41 %


,Category,Score,Reliability(%)
Rank,,,
1,Music,2651291836,100.00
2,Entertainment,2590088614,100.00
3,Sports,1366663953,33.33
4,Comedy,955594121,83.33
5,People & Blogs,878483088,86.67
6,Film & Animation,598235933,92.31
7,News & Politics,363101380,66.67
8,Howto & Style,335349083,90.91
9,Gaming,267110237,64.29


In [9]:
print('video\t\t\t','FR')
print('language\t\t',language,'\n')
print('Total reliablities')

statics = pd.DataFrame([],index=[1,2,3], 
                       columns=['sum_total_views','mean_total_views','number_of_unique_videos',
                                'sum_trending_duration','mean_trending_duration'])
statics.index.name = 'reliability_method'
for predict_method in statics.columns:
    result_data = cg.info_category()
    pre.score(result_data, stat, predict_method)
    for reliability_method in statics.index:
        result_reliability = result_data.copy()
        rs.calc(result_reliability, 'rank',kedoo_data[language].values,reliability_method)
        statics[predict_method][reliability_method] = np.mean(result_reliability['reliability'])
statics

video			 FR
language		 French 

Total reliablities


,sum_total_views,mean_total_views,number_of_unique_videos,sum_trending_duration,mean_trending_duration
reliability_method,,,,,
1,88.2353,74.5098,88.5621,89.5425,77.1242
2,82.4144,64.6054,83.0951,84.5323,70.7488
3,22.2222,11.1111,27.7778,38.8889,5.55556
